# 문제 1.

서울시 동별 아파트 매매거래 현황

In [22]:
import os
from src import mylib
import requests
import json

keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
key=mylib.getKey(keyPath)

_url='http://openAPI.seoul.go.kr:8088'
_key=str(key['dataseoul'])
_type='json'
_service='ListNecessariesPricesService'
_start_index=4001
_end_index=5000
_api="/".join([_url,_key,_type,_service,str(_start_index),str(_end_index)])
response = requests.get(_api).text

fp=open("NecessariesPrices_5.json",'a')
fp.write(response)

265416

In [33]:
import json

count = 0
for line in open('NecessariesPrices.json', 'r').readlines():
        dic=json.loads(line)
for i in dic['ListNecessariesPricesService']['row']:
    count += 1
print ("마지막줄 출력 : {}".format(dic['ListNecessariesPricesService']['row'][count-1]))
print("현재까지 수집한 건수 : {}".format(count))

마지막줄 출력 : {'P_SEQ': 1692651.0, 'M_SEQ': 98.0, 'M_NAME': '롯데백화점 노원점', 'A_SEQ': 306.0, 'A_NAME': '배(신고, 600g)', 'A_UNIT': '1개', 'A_PRICE': '10000', 'P_YEAR_MONTH': '2021-09', 'ADD_COL': '황금배', 'P_DATE': '2021-09-09', 'M_TYPE_CODE': '002', 'M_TYPE_NAME': '대형마트', 'M_GU_CODE': '350000', 'M_GU_NAME': '노원구'}
현재까지 수집한 건수 : 1000


In [8]:
!dir NecessariesPrices.json

 C 드라이브의 볼륨: OS
 볼륨 일련 번호: 9CB7-5096

 C:\Users\안성찬\workspace\201910810 디렉터리

2021-09-19  오전 08:43           290,655 NecessariesPrices.json
               1개 파일             290,655 바이트
               0개 디렉터리  36,675,903,488 바이트 남음


하루에 1000건씩, 총 312630건을 읽으려면 한달이 걸리지만 동별, 월별 차이이기 때문에 날짜의 범위를 좁히면 더 적게 추출하여 분석이 가능합니다. 

1000건에 283KB (290,655 바이트)이니 30000건을 저장해도 3MB 정도로 무리 없이 저장할 수 있습니다.

### *문제 2, 3 스파크 설정

In [1]:
import pyspark
import os
os.environ["PYSPARK_PYTHON"]='c:\\miniconda3\\python.exe'
os.environ["PYSPARK_DRIVER_PYTHON"]='c:\\miniconda3\\python.exe'

In [2]:
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

# 문제 2.

In [19]:
spark.version

'3.1.2'

In [10]:
!java --version

java 11.0.12 2021-07-20 LTS
Java(TM) SE Runtime Environment 18.9 (build 11.0.12+8-LTS-237)
Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.12+8-LTS-237, mixed mode)


In [11]:
!javac --version

javac 11.0.12


In [12]:
!python --version

Python 3.9.1


In [14]:
import sys
print(sys.version)

3.9.1 (default, Dec 11 2020, 09:29:25) [MSC v.1916 64 bit (AMD64)]


# 문제 3.

csv 파일은 ANSI로 인코딩 되어 있었다. 파일과 스파크 설정 사이의 인코딩 설정이 다르면 파일 깨짐이 발생할 수 있다. csv파일 인코딩 설정을 UTF-8로 하면 한글 깨짐 없이 출력할 수 있다.

## 1)

In [21]:
myRdd1=spark.sparkContext.textFile(os.path.join("data","경기도 의정부시_인구현황_20210910.csv"))

In [22]:
myRdd1.take(5)

['행정기관,인구수(계),인구수(남),인구수(여),구성비(계),구성비(남),구성비(여),성비,세대수,세대당인구,관리기관명,관리부서명,부서전화번호,데이터기준일자',
 '의정부1동,32292,16538,15754,6.97,3.57,3.4,104.98,19998,1.61,의정부시,민원여권과,031-828-2466,2021-09-10',
 '의정부2동,31380,15608,15772,6.77,3.37,3.4,98.96,16410,1.91,의정부시,민원여권과,031-828-2466,2021-09-10',
 '호원1동,36124,17595,18529,7.8,3.8,4,94.96,15653,2.31,의정부시,민원여권과,031-828-2466,2021-09-10',
 '호원2동,34957,16923,18034,7.54,3.65,3.89,93.84,13683,2.55,의정부시,민원여권과,031-828-2466,2021-09-10']

## 2)

In [24]:
myRdd2=spark.sparkContext.textFile(os.path.join("data","제주특별자치도 서귀포시_고령화비율및노령화지수현황_20210831.csv"))

In [25]:
myRdd2.take(5)

['연도별,서귀포시 인구수,65세이상 인구수 ,14세이하 인구수,고령화비율,노령화지수,데이터기준일자',
 '2012,154057,25826,22861,16.76,112.97,2021-08-31',
 '2013,155641,26936,22393,17.31,120.29,2021-08-31',
 '2014,158512,27877,22058,17.59,126.38,2021-08-31',
 '2015,164519,28979,22362,17.61,129.59,2021-08-31']